In [1]:
from calendar import month
from operator import index
import snscrape.modules.twitter as sntwitter
import pandas as pd
import glob
import datetime

release_dates = pd.read_csv("Release dates - data.csv")
release_dates_map = {}

folderpathrt = "rotten_info/*"
filesrt = []

for file in glob.glob(folderpathrt):
    filesrt.append(file)

for file in filesrt:
    print(file)
    movies = pd.read_csv(file)

    tweets = []
    for ind, movie in movies.iterrows():
        movien = movie['Release Group']
        if not pd.isna(movie['Release Date (Theaters)']):
            release_date = ('').join(movie['Release Date (Theaters)'].strip('[]').split(',')).replace("'", "")
            rd = datetime.datetime.strptime(release_date, "%b %d %Y")
            # if release_dates_map.get(movien.lower(), ''):
            release_dates_map[movien.lower()] = rd

folderpath = "Box Office Mojo/*"
files = []

for file in glob.glob(folderpath):
    files.append(file)

for ind, row in release_dates.iterrows():
    if row['Release-Date']:
        rd = datetime.datetime.strptime(row['Release-Date'], "%Y-%m-%d")
        if not release_dates_map.get(row['Movie'].lower(), None):
            release_dates_map[row['Movie'].lower()] = rd
    
count = 0
c = 0
for file in files:
    if '2021' in file:
        print(file)
        movies = pd.read_csv(file)

        tweets = []
        for ind, movie in movies.iterrows():
            c += 1
            query_txt = movie['Release Group']
            release_date = release_dates_map.get(movie['Release Group'].lower(), None)
            if release_date:
                count += 1
                # rd = datetime.datetime.strptime(release_date, "%Y-%m-%d")
                since = release_date - datetime.timedelta(weeks=4)
                until = release_date + datetime.timedelta(weeks=1)
                query_txt += " since:%s until:%s" % (since.date(), until.date())
                print(movie['Release Group'], release_date.date())

                for i,tweet in enumerate(sntwitter.TwitterSearchScraper(query_txt).get_items()):
                    if i>5000:
                        break
                    tweets.append([movie['Release Group'], tweet.date, tweet.id, tweet.content, tweet.user.username])

        tweets_df = pd.DataFrame(tweets, columns=['Movie Name', 'Datetime', 'Tweet Id', 'Text', 'Username'])
        tweets_df.to_csv("Twitter/tweets_%s" % file.split('/')[1], index=False, escapechar=r'|')
        break


Box Office Mojo/2021.csv
Spider-Man: No Way Home 2021-12-17
The Battle at Lake Changjin 2021-11-19
No Time to Die 2021-10-08
F9: The Fast Saga 2021-06-25
Venom: Let There Be Carnage 2021-10-01
Godzilla vs. Kong 2021-03-31
Shang-Chi and the Legend of the Ten Rings 2021-09-03
Eternals 2021-11-05
Dune 2021-10-22
Black Widow 2021-07-09
Sing 2 2021-12-22
Free Guy 2021-08-13
A Quiet Place Part II 2021-05-28
Encanto 2021-11-24
Cruella 2021-05-28
Jungle Cruise 2021-07-30
Raging Fire 2021-08-13
The Conjuring: The Devil Made Me Do It 2021-06-04
Ghostbusters: Afterlife 2021-11-19
The Suicide Squad 2021-08-06
Space Jam: A New Legacy 2021-07-16
House of Gucci 2021-11-24
The Matrix Resurrections 2021-12-22
Peter Rabbit 2: The Runaway 2021-06-11
The Boss Baby 2: Family Business 2021-07-02
PAW Patrol: The Movie 2021-08-20
Tom & Jerry: The Movie 2021-02-26
Jujutsu Kaisen 0: The Movie 2022-03-18
Halloween Kills 2021-10-15
Raya and the Last Dragon 2021-03-05
The King's Man 2021-12-22
The Addams Family 2 